# Receiving robot state

In this notebook, we will demonstrate how to receive and process state messages from the UR3e mockup. The mockup publishes its state messages with the `robotarm.state` routing key:
- `ROBOT_MODE`: Current mode of the robot (e.g., RUNNING, Idle).
- `Q_ACTUAL`: Current joint positions (radians).
- `QD_ACTUAL`: Current joint velocities (radians/second).
- `Q_TARGET`: Target joint positions (radians).
- `TIMESTAMP`: Timestamp of the status message.
- `JOINT_MAX_SPEED`: Maximum joint speeds (radians/second).
- `JOINT_MAX_ACCELERATION`: Maximum joint accelerations (radians/second²).
- `TCP_POSE`: Current TCP pose (position and orientation).

To receive these messages, we will set up a RabbitMQ consumer that listens for messages with the `robotarm.state` routing key. Below is an example of how to implement this in Python using the communication module provided in the project.

In [ ]:
from communication import protocol
from communication.rabbitmq import Rabbitmq

try:
    rmq = Rabbitmq(
        ip="localhost",
        port=5672,
        username="incubator",
        password="incubator",
        vhost="/",
        exchange="UR3E_AMQP",
        type="topic",
    )
    rmq.connect_to_server()
    print("✓ Connected to RabbitMQ successfully")
except Exception as e:
    print(f"✗ Failed to connect to RabbitMQ: {e}")
    print("\nMake sure RabbitMQ is running. You can start it with:")
    print("  python -m startup.start_docker_rabbitmq")

# define a callback function to handle received messages
def on_message_received(ch, method, properties, body):
    try:
        print("✓ State:")
        print(body)
    except Exception as e:
        print(f"✗ Failed to decode the message: {e}")

Running the next cell will set up the subscription and start receiving state messages from the mockup. Note that this will block the notebook until interrupted, which is in

In [ ]:
rmq.subscribe(
    routing_key=protocol.ROUTING_KEY_STATE,
    on_message_callback=on_message_received,
)

rmq.start_consuming()

Try to modify the callback function to log the received state messages in a more structured format, such as a csv file. 

In [ ]:
rmq.close()